In [1]:
import os
import platform
import psutil

# CPUの基本情報を取得
cpu_name = platform.processor()
cpu_arch = platform.architecture()
cpu_core_count = os.cpu_count()

print(f"CPU Name: {cpu_name}")
print(f"CPU Architecture: {cpu_arch[0]}")
print(f"Number of CPU Cores: {cpu_core_count}")

# psutilを使用してCPUの詳細情報を取得
cpu_freq = psutil.cpu_freq()
cpu_percent = psutil.cpu_percent(interval=1)
virtual_memory = psutil.virtual_memory()

print(f"CPU Max Frequency: {cpu_freq.max}Mhz")
print(f"CPU Current Frequency: {cpu_freq.current}Mhz")
print(f"CPU Utilization: {cpu_percent}%")
print(f"Total Virtual Memory: {virtual_memory.total / (1024 ** 3):.2f} GB")



CPU Name: x86_64
CPU Architecture: 64bit
Number of CPU Cores: 32
CPU Max Frequency: 4937.5Mhz
CPU Current Frequency: 2874.5446249999995Mhz
CPU Utilization: 6.9%
Total Virtual Memory: 62.53 GB


### 1. test

In [2]:
import requests

input_data = "hello"

def query(input_data):
    response = requests.post('http://test.advml.com/score', json={'data': input_data})
    return response.json()

query(input_data)

{'flag': 'gAAAAABlP6Ta8DfQOU5wIwkHmQESKGpSFOAtRXSeUjDoD4LwxYn0H7X3e9klLB-wHaiwhalQ9pAfxKo3tjhEO-MEYQg9rUDwjx2R6tDK-Q7YZPxvjS48i_qW191rwarLyS5uahDtcuhIMnCKIhWCwVzQ1aTIv6rmYDUId_uZrp1Y0woZEedGJLM='}

## 2. Cluster1

In [19]:
import sklearn
import skops
print(sklearn.__version__)
print(skops.__version__)

1.3.0
0.9.0


In [4]:
with open("./cluster1/census_model.skops", 'rb') as f:
    model = f.read()
    
with open("./cluster1/census.csv", 'r') as f:
    data = f.read()

In [14]:
import requests

input_data = [0]

def query(input_data):
    response = requests.post('http://cluster1.advml.com/score', json={'data': input_data})
    return response.json()

query(input_data)

{'message': 'Too small! groups of <1% dataset size are not representative. Score: 0'}

In [42]:
import pandas as pd

# CSVファイルのパス
file_path = "./cluster1/census.csv"

# CSVファイルを読み込む
df = pd.read_csv(file_path)

In [25]:
df[["id", "education.num"]]

,id,education.num
0,19956,10
1,9864,10
2,27689,4
3,32524,9
4,11365,10
...,...,...
16276,17199,10
16277,25421,13
16278,107,10
16279,10911,13


In [26]:
sub_df = df[["id", "education.num"]].copy()

In [30]:
df = df.drop("education.num", axis=1)

In [31]:
df

,id,age,workclass,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,19956,23,Private,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K
1,9864,43,Private,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,>50K
2,27689,49,Self-emp-not-inc,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,<=50K
3,32524,33,Private,Married-civ-spouse,Craft-repair,Husband,Black,Male,0,0,40,United-States,<=50K
4,11365,22,Private,Never-married,Adm-clerical,Own-child,White,Male,0,0,25,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,17199,23,Private,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K
16277,25421,49,Self-emp-not-inc,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,70,United-States,<=50K
16278,107,54,Private,Married-civ-spouse,Sales,Husband,White,Male,0,2415,50,United-States,>50K
16279,10911,30,Self-emp-inc,Married-civ-spouse,Sales,Husband,White,Male,0,0,52,United-States,>50K


In [22]:
from skops.io import load
model = load("./cluster1/census_model.skops", trusted=True)
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OrdinalEncoder(),
                                                  ['workclass', 'education.num',
                                                   'marital.status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native.country']),
                                                 ('num', 'passthrough',
                                                  ['age', 'capital.gain',
                                                   'capital.loss',
                                                   'hours.per.week'])])),
                ('classifier',
                 HistGradientBoostingClassifier(categorical_features=array([0, 1, 2, 3, 4, 5, 6, 7]),
                                                learning_rate=0.01, max_depth=5,
                                                max_iter=5000))])

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              16281 non-null  int64 
 1   age             16281 non-null  int64 
 2   workclass       15349 non-null  object
 3   marital.status  16281 non-null  object
 4   occupation      15345 non-null  object
 5   relationship    16281 non-null  object
 6   race            16281 non-null  object
 7   sex             16281 non-null  object
 8   capital.gain    16281 non-null  int64 
 9   capital.loss    16281 non-null  int64 
 10  hours.per.week  16281 non-null  int64 
 11  native.country  15981 non-null  object
 12  income          16281 non-null  object
dtypes: int64(5), object(8)
memory usage: 1.6+ MB


In [40]:
# conversion of categorical features to numerical features
from sklearn.preprocessing import OrdinalEncoder

df_category = df[["workclass", "marital.status", "occupation", "relationship", "race", "sex", "native.country", "income"]]
oe = OrdinalEncoder()
oe.fit(df_category)
df_trans = oe.transform(df_category)
df[["workclass", "marital.status", "occupation", "relationship", "race", "sex", "native.country", "income"]] = df_trans


In [41]:
# 

,id,age,workclass,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,19956,23,3.0,4.0,0.0,1.0,4.0,0.0,0,0,40,37.0,0.0
1,9864,43,3.0,2.0,3.0,0.0,4.0,1.0,0,0,50,37.0,1.0
2,27689,49,5.0,2.0,11.0,0.0,4.0,1.0,0,0,40,37.0,0.0
3,32524,33,3.0,2.0,2.0,0.0,2.0,1.0,0,0,40,37.0,0.0
4,11365,22,3.0,4.0,0.0,3.0,4.0,1.0,0,0,25,37.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,17199,23,3.0,2.0,13.0,0.0,4.0,1.0,0,0,40,37.0,0.0
16277,25421,49,5.0,2.0,4.0,0.0,4.0,1.0,0,0,70,37.0,0.0
16278,107,54,3.0,2.0,11.0,0.0,4.0,1.0,0,2415,50,37.0,1.0
16279,10911,30,4.0,2.0,11.0,0.0,4.0,1.0,0,0,52,37.0,1.0


In [39]:
import numpy as np 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

array([[ 3.,  4.,  0., ...,  0., 37.,  0.],
       [ 3.,  2.,  3., ...,  1., 37.,  1.],
       [ 5.,  2., 11., ...,  1., 37.,  0.],
       ...,
       [ 3.,  2., 11., ...,  1., 37.,  1.],
       [ 4.,  2., 11., ...,  1., 37.,  1.],
       [ 3.,  4.,  5., ...,  1., 37.,  0.]])

In [43]:
df # 

,id,age,workclass,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,19956,23,Private,10,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K
1,9864,43,Private,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,>50K
2,27689,49,Self-emp-not-inc,4,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,<=50K
3,32524,33,Private,9,Married-civ-spouse,Craft-repair,Husband,Black,Male,0,0,40,United-States,<=50K
4,11365,22,Private,10,Never-married,Adm-clerical,Own-child,White,Male,0,0,25,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,17199,23,Private,10,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K
16277,25421,49,Self-emp-not-inc,13,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,70,United-States,<=50K
16278,107,54,Private,10,Married-civ-spouse,Sales,Husband,White,Male,0,2415,50,United-States,>50K
16279,10911,30,Self-emp-inc,13,Married-civ-spouse,Sales,Husband,White,Male,0,0,52,United-States,>50K
